<a href="https://colab.research.google.com/github/shelleyg-bit/canada-land-cover-classifier/blob/main/gee_sentinel_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Earth engine and Initialize it

In [ ]:
!pip install geopandas
!pip install geojson
!pip install geemap
!pip install geotable

     |████████████████████████████████| 130 kB 44.5 MB/s 
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=e61022981c2e1df57f7242450382419622405e239a3b0eedf21568770504ad5b
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm


In [ ]:
"""
Created on Apr 30, 2022
@author: Mani Babu

Purpose: convert kmz and excel file to shape files 
"""
import pandas as pd
import geopandas as gpd
import geotable
from shapely.geometry import Point, Polygon
from pyproj import CRS

from pathlib import Path
import os
import matplotlib.pyplot as plt
import json


import warnings
warnings.filterwarnings('ignore')

def kmz_2_df(path, shape =False):
    '''
    Convert KMZ files to geopandas dataframe
    Args: 
        path - path for kmz files
    
    '''
    
    kmz_data =pd.DataFrame()
    for file in os.listdir(path):
        if '.kmz' in file:
            data = geotable.load(path+file)
            kmz_data =kmz_data.append(data)
    kmz_data.rename(columns ={'geometry_object': 'geometry'}, inplace =True)
    kmz_data_sub =kmz_data[['Name', 'geometry']].reset_index().drop('index', axis=1)
    kmz_data_sub = gpd.GeoDataFrame(kmz_data_sub)
    kmz_data_sub.crs =CRS.from_epsg(4326).to_wkt()
    
    if shape:
        kmz_data_sub.to_file(str(Path(path).parents[0])+'\shape')
        
    return kmz_data_sub, kmz_data


def excel_2_df(path, sheet, shape=False):
    '''
    Convert excel data to geopandas dataframe
    Args: 
        path - path for excel file including the file name
    
    '''    
    excel_data =pd.read_excel(path, sheet ,header=1, usecols="A:G")
    excel_data.dropna(inplace =True)
    excel_data['field_boundary1'] =excel_data['field_boundary'].apply(lambda x: json.loads(x))
    excel_data['geometry'] = excel_data['field_boundary1'].apply(lambda x: Polygon(x['coordinates'][0][0]))
    excel_data_sub =gpd.GeoDataFrame(excel_data[['farm_name', 'geometry' ]])
    excel_data_sub.rename(columns ={'farm_name': 'Name' }, inplace =True)
    excel_data_sub.crs =CRS.from_epsg(4326).to_wkt()
    if shape:
        excel_data_sub.to_file(str(Path(path).parents[0])+'\shape')
    
    return excel_data_sub, excel_data


def comb_kmz_n_excel_2shp(path1, path2, sheet):
    '''
    Convert both KMZ and excel files to shape files and add cordinate system.
    Args: 
        path1 - path for kmz files
        path2 - path for excel file including the file name
        sheet - sheet to load
    
    '''
    
    df1,_ = kmz_2_df(path1)
    df2,_ =excel_2_df(path2, sheet)
    df_final =pd.concat([df2,df1]).reset_index().drop('index', axis=1)
    df_final.crs =CRS.from_epsg(4326).to_wkt()
    df_final.to_file(str(Path(path1).parents[0])+'\shape')
    return df_final

In [ ]:
import ee
ee.Authenticate() # give this notebook access to your google ee account
ee.Initialize() # this talks to ee backend to find out about all features of ee

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=MmAXnadBGVUh3g4BkpiqiB1Wtf-un3XczvVCZqH84oQ&tc=eLCFPpDpIzw4KZwPVCaAu3Qn1SG1vimGOcD_nOXP28Q&cc=TaXWHhAN_4eheAsxdGf0ts0isdnA0Wfvp1eAtreWacc

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh9_gBFG1DOi0dzZ1_A1yeK8xxPRZIV9hx1Lte7V-SbdLqlOlTbemE

Successfully saved authorization token.


## Define an earth engne image object from satellite data
[Link to GEE data catalog](https://developers.google.com/earth-engine/datasets/catalog)

[Note] before mounting drive, first add shortcut of geometries folder present in gdrive farmhand data folder to MyDrive

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
geometry_files_path = '/drive/MyDrive/farmhand/Data' 

In [ ]:
!unzip /drive/MyDrive/farmhand/Data/All\ Maatrabhumi\ Kml\ files_farms\ not\ on\ FH\ Platform_sheet\ 1.zip


In [ ]:
kmz_dirpath = 'All MB kml files/'
#farms_geom_df = comb_kmz_n_excel_2shp(kmz_dirpath, '/drive/MyDrive/farmhand/Data/farms_sheet.xlsx', 'Farm data (for farms created in platform')
# farms_geom_df = kmz_2_df(kmz_dirpath)[0]
farms_geom_df = excel_2_df('/drive/MyDrive/farmhand/Data/farms_sheet.xlsx', 'Farm data (for farms created in')[0]


In [ ]:
list(farms_geom_df['geometry'][0].exterior.coords)
first_bounds = farms_geom_df['geometry'][0].bounds
bbox = ee.Geometry.BBox(*first_bounds)
for farm in farms_geom_df['geometry']:
    bounds = farm.bounds
    print(bounds)
    bbox = bbox.union(ee.Geometry.BBox(*bounds), maxError=1)

(76.9695158337162, 15.1110569315164, 76.9705432005502, 15.1135470713739)
(76.9746738707725, 15.1175489465957, 76.9751980958454, 15.119004425874)
(76.9746984090336, 15.1190740968918, 76.9752656116175, 15.1203214269165)
(76.9748199468573, 15.1203606227025, 76.9753553813936, 15.1213119508452)
(76.9785147491029, 15.1067101786961, 76.9810740401433, 15.1081006344757)
(76.9784777280811, 15.1058413116665, 76.9810109917785, 15.1067530053938)
(76.9789452840366, 15.1208546549711, 76.9819938916949, 15.1225700019219)
(76.9718632618108, 15.1107892064052, 76.9738470505863, 15.1115754677288)
(76.7380359453208, 15.2939712487795, 76.7386301909456, 15.297143073852)
(76.7264758824325, 15.2996276729674, 76.7277202281259, 15.3021339535692)
(76.9775985298879, 15.1209009002315, 76.9787387749585, 15.1217554983129)
(76.7226482866705, 15.2435108323871, 76.7232061229779, 15.2448766873416)
(76.7179872, 15.2470315353301, 76.7188302, 15.2490654)
(76.7226540060802, 15.2435455088052, 76.7230184385276, 15.2446153910597

In [ ]:
bbox.area(maxError=1).getInfo()

2522691.3211143008

In [ ]:
type(farms_geom_df)
farms_geom_df.head()

,Name,geometry
0,Anantha lakshmi K,"POLYGON ((76.97053 15.11344, 76.96976 15.11355..."
1,Anjamma K,"POLYGON ((76.97472 15.11900, 76.97520 15.11896..."
2,Anjamma K,"POLYGON ((76.97470 15.11925, 76.97479 15.11910..."
3,Anjamma K,"POLYGON ((76.97530 15.12040, 76.97508 15.12036..."
4,Anji Babu,"POLYGON ((76.98107 15.10781, 76.98096 15.10671..."


In [ ]:
farms_geom_df.crs = CRS.from_epsg(4326).to_wkt()

In [ ]:
farms_roi = geemap.geopandas_to_ee(farms_geom_df)

In [ ]:

import geopandas as gpd
import folium 
import geojson

import numpy as np

import matplotlib.pyplot as plt
from skimage import data
from skimage.color import rgb2gray
from skimage.io import imread
from skimage import exposure
from skimage.filters import try_all_threshold
from skimage.filters import threshold_otsu, threshold_local
from skimage import measure
from skimage import feature


import IPython.display as disp
%matplotlib inline


In [ ]:
from zipfile import ZipFile

In [ ]:
from xml.dom import minidom
import pandas as pd

 # Prepare Geometry

In [ ]:
import geemap
import os

def kmz_to_kml(kmz_filepath, kml_filepath):
    """Function to extract kmz file
    amd read its XML part
    to write to kml file

    Credit: Adopted from Aaron CL's script 
    https://dagshub.com/Omdena/FarmHand/src/data_collection/preprocessing_files.py
    """
    zf = ZipFile(kmz_filepath, 'r')
    kml_doc = zf.namelist()[0]
    content = zf.read(kml_doc)
    xml_doc = minidom.parseString(content).toxml()
    with open(kml_filepath, 'w') as kml_file:
        kml_file.writelines(xml_doc)

kmz_dirpath = 'All MB kml files'
farms_geom_lst = []
for file in os.listdir(kmz_dirpath):
    if file.endswith('kmz'):
        kmz_filepath = kmz_dirpath + '/' + file
        kml_temp_filepath = kmz_dirpath + '/temp.kml'
        kmz_to_kml(kmz_filepath, kml_temp_filepath)
        farms_geom_lst.append(gpd.read_file(kml_temp_filepath))

farms_geom_df = pd.concat(farms_geom_lst, ignore_index=True)
farms_geom_df.rename(columns={'Name':'id'}, inplace=True)
farms_geom_df.drop(columns=['Description'], inplace=True)

In [ ]:
farms_geom_df.head()
geojson.loads(farms_geom_df['geometry'][0])

TypeError: ignored

In [ ]:
list(farms_geom_df['geometry'][0].boundary.coords)

[(76.71314422041178, 15.25548987615254, 0.0),
 (76.7132555320859, 15.25651945223919, 0.0),
 (76.7126323, 15.2565541, 0.0),
 (76.7125404, 15.2554993, 0.0),
 (76.71314422041178, 15.25548987615254, 0.0)]

In [ ]:
gdf = gpd.read_file(
    "https://raw.githubusercontent.com/giswqs/data/main/us/us_states.geojson"
)
gdf.head()

,id,name,geometry
0,AL,Alabama,"MULTIPOLYGON (((-87.35930 35.00118, -85.60667 ..."
1,AK,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,AZ,Arizona,"MULTIPOLYGON (((-109.04250 37.00026, -109.0479..."
3,AR,Arkansas,"MULTIPOLYGON (((-94.47384 36.50186, -90.15254 ..."
4,CA,California,"MULTIPOLYGON (((-123.23326 42.00619, -122.3788..."


In [ ]:
fc = geemap.geopandas_to_ee(farms_geom_df)

In [ ]:
type(farms_geom_df['geometry'][0])
print(type(farms_geom_df))
geemap.geopandas_to_ee(farms_geom_df[0])

<class 'geopandas.geodataframe.GeoDataFrame'>


KeyError: ignored

In [ ]:
collection_sentinel = ('COPERNICUS/S2_SR')
time_range = ['2019-08-15', '2022-03-15']

In [ ]:
def obtain_image_sentinel_median(collection, time_range, area):
    """ Selection of median, cloud-free image from a collection of images in the Sentinel 2 dataset
    See also: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        sentinel_median (ee.image.Image)
     """
    # method to remove cloud from the image
    def maskclouds(image):
        """To mask clouds using the Sentinel-2 QA band
        @param {ee.Image} image Sentinel-2 image
        @return {ee.Image} cloud masked Sentinel-2 image
        """
        band_qa = image.select('QA60')
        cloud_mask = ee.Number(2).pow(10).int()
        cirrus_mask = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
            band_qa.bitwiseAnd(cirrus_mask).eq(0))
        return image.updateMask(mask).divide(10000)

    sentinel_filtered = (ee.ImageCollection(collection).
                         filterBounds(area).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)).
                         map(maskclouds))

    sentinel_median = sentinel_filtered.median().clip(area) # TODO: change median to get time series data
    return sentinel_median

def obtain_image_sentinel_allbands(collection, time_range, area):
    """ Selection of median, cloud-free image from a collection of images in the Sentinel 2 dataset
    See also: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        sentinel_median (ee.image.Image)
     """
    # method to remove cloud from the image
    def maskclouds(image):
        """To mask clouds using the Sentinel-2 QA band
        @param {ee.Image} image Sentinel-2 image
        @return {ee.Image} cloud masked Sentinel-2 image
        """
        band_qa = image.select('QA60')
        cloud_mask = ee.Number(2).pow(10).int()
        cirrus_mask = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
            band_qa.bitwiseAnd(cirrus_mask).eq(0))
        return image.updateMask(mask).divide(10000)

    sentinel_filtered = (ee.ImageCollection(collection).
                         filterBounds(area).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)).
                         map(maskclouds))

    sentinel_median = sentinel_filtered.select(['B8', 'B4', 'B3']) # TODO: change median to get time series data
    return sentinel_median

In [ ]:
bbox = farms_roi.bounds()

AttributeError: ignored

In [ ]:
farms_images_b8_4_3 = obtain_image_sentinel_allbands(collection_sentinel, time_range, area=farms_roi)
geemap.ee_export_image(farms_images_b8_4_3.first(), 'MyDrive/sentinel.tif', region=farms_roi.getInfo()['features'][0]['geometry']['coordinates'], crs='EPSG:4326', file_per_band=1)

Generating URL ...
Please wait ...
Data downloaded to /content/MyDrive


In [ ]:
#bellari_allbands.first().getInfo()['bands']
print(bellari_allbands.first().projection().getInfo())
#print(area_bellari.projection())
reproj = bellari_allbands.first().reproject(crs=area_bellari.projection().getInfo()['crs'])
print(reproj.projection().getInfo())


{'type': 'Projection', 'crs': 'EPSG:32643', 'transform': [10, 0, 499980, 0, -10, 1700040]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [ ]:
bellari_allbands.first().getDownloadURL({'region': (area_bellari), 'scale':(30)})

In [ ]:
type(bellari_allbands)

ee.imagecollection.ImageCollection

# Visualizing Satellite Imagery

[Indices & composities](https://github.com/sentinel-hub/custom-scripts/tree/master/sentinel-2/false_color_infrared)

[Gee guide for visualization parameters](https://tutorials.geemap.org/Image/image_visualization/)

In [ ]:
import geemap


In [ ]:
location = area_bellari.getInfo()['features'][0]['geometry']['coordinates'][0][0][::-1]

In [ ]:
location

[14.886980056000084, 76.67508697500006]

In [ ]:

#location = area_bellari.centroid().coordinates().getInfo()[::-1] # In coordinates first number is longitude
Map = geemap.Map(center=location, zoom=8)
#Map.addLayer(area_bellari, {}, 'bellari_geom', opacity=0.5)
#bellari_bounds = area_bellari.bounds()
#Map.addLayer(bellari_bounds, {'color':'black'}, 'Bounding Box')

In [ ]:
bellari_bounds.getInfo()

{'coordinates': [[[75.664612, 14.556279999999973],
   [77.165253, 14.556279999999973],
   [77.165253, 15.829490000000026],
   [75.664612, 15.829490000000026],
   [75.664612, 14.556279999999973]]],
 'geodesic': False,
 'type': 'Polygon'}

In [ ]:
vis_params = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0.2,
    'max': 0.9
}

# my_map = folium.Map(location=location, zoom_start=10)

Map.addLayer(bellari_median, vis_params, 'sentinel false color infrared for vegetation ') 



In [ ]:
Map.addLayer(farms_roi)


In [ ]:
convexhull = bbox.convexHull(maxError=1)

In [ ]:
Map.addLayer(bbox, {'color': 'black'})

In [ ]:
Map.addLayer(convexhull)

In [ ]:
Map.addLayer(convexhull.bounds(), {'color': 'cyan'})

In [ ]:
bbox2 = ee.Geometry(convexhull.bounds().getInfo())

In [ ]:
bbox2.bounds()

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.bounds",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    76.706359228535,
                    15.102785878010577
                  ],
                  [
                    76.9861363073877,
                    15.102785878010577
                  ],
                  [
                    76.9861363073877,
                    15.302133954429165
                  ],
                  [
                    76.706359228535,
                    15.302133954429165
                  ],
                  [
                    76.706359228535,
                    15.102785878010577
                  ]
                ]
              ]
            },
            "geodesic": {
              "c

In [ ]:
Map

Map(bottom=958807.0, center=[15.277064633885882, 76.73924446105958], controls=(WidgetControl(options=['positio…

In [ ]:
bellari_allbands.getInfo()

In [ ]:
bellari_median.getInfo()['bands']

In [ ]:
type(a)

ee.image.Image

In [ ]:
a.getInfo()['bands']

[{'crs': 'EPSG:4326',
  'crs_transform': [1, 0, 0, 0, 1, 0],
  'data_type': {'max': 6.553500175476074,
   'min': 0,
   'precision': 'float',
   'type': 'PixelType'},
  'dimensions': [3, 3],
  'id': 'B1',
  'origin': [75, 14]},
 {'crs': 'EPSG:4326',
  'crs_transform': [1, 0, 0, 0, 1, 0],
  'data_type': {'max': 6.553500175476074,
   'min': 0,
   'precision': 'float',
   'type': 'PixelType'},
  'dimensions': [3, 3],
  'id': 'B2',
  'origin': [75, 14]}]

In [ ]:
coords = bellari_gj['geometry']['coordinates'][0]
area_bellari = ee.Geometry.Polygon(coords[0])

In [ ]:
coords_bbox = bellari_bounds.getInfo()['coordinates'][0]
coords_bbox
west = coords_bbox[0][0]
east = coords_bbox[1][0]
south = coords_bbox[0][1]
north = coords_bbox[2][1]
print(west, east, south, north)
west1 = west
east1 = west + (east - west)/2
east2 = east


75.664612 77.165253 14.556279999999973 15.829490000000026


In [ ]:
bbox = ee.Geometry.BBox(west, south, east, north)

In [ ]:
area_bellari.get(0).getInfo()

EEException: ignored

In [ ]:
bbox.area(maxError=1).getInfo()
bbox_split = bbox.cutLines(distances=[0.001]*3, maxError=1)

In [ ]:
bbox_split.getInfo()

{'coordinates': [], 'type': 'MultiLineString'}

In [ ]:
geemap.ee_export_image(bellari_allbands.select('20220312T051651_20220312T052910_T43PFT_B4'), 'MyDrive/bellari_geometry/stest.tif', file_per_band=1, region=area_bellari)

Generating URL ...
Please wait ...
An error occurred while downloading.


JSONDecodeError: ignored

In [ ]:
len(bellari_allbands.getInfo()['bands'])

264